In [10]:
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher
from traccuracy.matchers._base import Matched, Matcher
from traccuracy.metrics import CTCMetrics, DivisionMetrics
from traccuracy._tracking_graph import TrackingGraph
from traccuracy import run_metrics
import h5py
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import networkx as nx
from tqdm import tqdm

In [11]:
class CTC_Centroid_Matcher(Matcher):
    """Match graph nodes based on measure used in cell tracking challenge benchmarking.

    A computed marker (segmentation) is matched to a reference marker if the computed
    marker covers a majority of the reference marker.

    Each reference marker can therefore only be matched to one computed marker, but
    multiple reference markers can be assigned to a single computed marker.

    See https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0144959
    for complete details.
    """

    def __init__(self, distance_threshold, scale=(1.0, 1.0, 1.0)):
        # distance_threshold (float): Maximum distance between centroids to be considered a match
        # scale (tuple): scaling factor for the x, y, z dimensions (anisotropy)

        # note that distance_threshold is in real isotropic units
        self.distance_threshold = distance_threshold
        self.scale = scale

    def _compute_frame_mapping(
        self, frame: int, gt_graph, pred_graph
    ):
        """Compute the distance matrix between centroids of gt and pred nodes"""
        gt_nodes = np.array(list(gt_graph.nodes_by_frame[frame]))
        pred_nodes = np.array(list(pred_graph.nodes_by_frame[frame]))

        gt_locs = (
            np.array([gt_graph.get_location(node) for node in gt_nodes]) * self.scale
        )
        pred_locs = (
            np.array([pred_graph.get_location(node) for node in pred_nodes])
            * self.scale
        )
        distance_matrix = cdist(gt_locs, pred_locs)

        row_ind, col_ind = linear_sum_assignment(distance_matrix)
        distances = distance_matrix[row_ind, col_ind]
        is_valid = distances <= self.distance_threshold

        gt_nodes = gt_nodes[row_ind[is_valid]]
        pred_nodes = pred_nodes[col_ind[is_valid]]

        return gt_nodes, pred_nodes, distances[is_valid]
    
    def _compute_mapping(self, gt_graph, pred_graph):
        """Run ctc matching

        Args:
            gt_graph (TrackingGraph): Tracking graph object for the gt
            pred_graph (TrackingGraph): Tracking graph object for the pred

        Returns:
            traccuracy.matchers.Matched: Matched data object containing the CTC mapping
        """
        mapping = []
        # Get overlaps for each frame
        for t in tqdm(
            range(gt_graph.start_frame, gt_graph.end_frame),
            desc="Matching frames",
        ):
            print(gt_graph)
            print(pred_graph)
            gt_match, pred_match, _ = self._compute_frame_mapping(
                t, gt_graph, pred_graph
            )
            for gt_node, pred_node in zip(gt_match, pred_match):
                mapping.append((gt_node, pred_node))

        return Matched(gt_graph, pred_graph, mapping)

In [12]:
def evaluate_tracks(tracking_graph, gt_tracking_graph, distance_threshold, scale):
    metrics = run_metrics(
        gt_tracking_graph,
        tracking_graph,
        matcher=CTC_Centroid_Matcher(distance_threshold, scale),
        metrics=[CTCMetrics()],
    )
    return metrics

In [13]:
def tracks_to_tracking_graph(tracks):
    """
    Convert calcium segmentation data into a traccuracy TrackingGraph.

    Parameters:
    tracks (numpy array): Array with shape (1500, 118, 3) containing
                          the time (first dimension), objects (second dimension),
                          and XYZ coordinates (third dimension).

    Returns:
    traccuracy.TrackingGraph: A tracking graph built from the provided calcium segmentation data.
    """
    graph = nx.DiGraph()
    num_time_points, num_objects, _ = tracks.shape

    # Create unique identifiers for each object at each time point
    object_ids = {(t, obj): f"object_{t}_{obj}" for t in range(num_time_points) for obj in range(num_objects)}

    # Add nodes to the graph
    for t in range(num_time_points):
        for obj in range(num_objects):
            x, y, z = tracks[t, obj]
            graph.add_node(object_ids[(t, obj)], x=x, y=y, z=z, t=t)

    # Add edges to the graph
    for t in range(num_time_points - 1):
        for obj in range(num_objects):
            current_id = object_ids[(t, obj)]
            next_id = object_ids[(t + 1, obj)]
            if current_id in graph.nodes and next_id in graph.nodes:
                graph.add_edge(current_id, next_id)

    tracking_graph = TrackingGraph(graph, location_keys=("x", "y", "z"))

    return tracking_graph

In [25]:
dataset_id2xyres = {
        "EY": 0.27,
        "KK": 0.32,
        "SK1": 0.3208,
        "SK2": 0.3208,
    }
dataset_id2zres = {
        "EY": 1.5,
        "KK": 1.5,
        "SK1": 2.5,
        "SK2": 1.5,
    }
dataset_id='EY'
xyz_resol = (dataset_id2xyres[dataset_id], dataset_id2xyres[dataset_id], dataset_id2zres[dataset_id], )

file_pairs = [
    ('sub-20190929-06_reordered_seg20_centroids.npy', 'sub-20190929-06_ses-20190929_ophys.h5'),
    ('sub-20191030-07_reordered_seg20_centroids.npy', 'sub-20191030-07_ses-20191030_ophys.h5'),
    ('sub-20190928-11_reordered_seg20_centroids.npy', 'sub-20190928-11_ses-20190928_ophys.h5'),
    ('sub-20190928-03_reordered_seg20_centroids.npy', 'sub-20190928-03_ses-20190928_ophys.h5')
]

metrics3_list = []
metrics6_list = []

for np_path, h5_path in file_pairs:
    f = h5py.File(h5_path, 'r')
    calcium_image = f['calcium_image'][:]
    calcium_image = np.transpose(calcium_image, (0, 3, 2, 1)) # ultrack takes (t, z, y, x)
    calcium_segmentation = f['calcium_segmentation'][:] # t, xyz
    
    
    pred_data = np.load(np_path)
    pred_data = np.nan_to_num(pred_data, nan=0)
    
    print(f"Processing {np_path} and {h5_path}:")
    print(pred_data.shape)
    print(calcium_segmentation.shape)
    
    pred_track = tracks_to_tracking_graph(pred_data)
    gt_track = tracks_to_tracking_graph(calcium_segmentation)
    
    metrics3 = evaluate_tracks(pred_track, gt_track, 3, xyz_resol)[0]
    metrics6 = evaluate_tracks(pred_track, gt_track, 6, xyz_resol)[0]

    metrics3_list.append(metrics3)
    metrics6_list.append(metrics6)

print("Metrics3 for all file pairs:", metrics3_list)
print("Metrics6 for all file pairs:", metrics6_list)

Processing sub-20190929-06_reordered_seg20_centroids.npy and sub-20190929-06_ses-20190929_ophys.h5:
(962, 400, 3)
(961, 178, 3)


Matching frames:   4%|▍         | 37/961 [00:00<00:05, 184.20it/s]

Matching frames:   8%|▊         | 80/961 [00:00<00:04, 204.06it/s]

Matching frames:  11%|█         | 103/961 [00:00<00:04, 211.78it/s]

Matching frames:  15%|█▌        | 148/961 [00:00<00:03, 217.34it/s]

...

Matching frames:  88%|████████▊ | 849/961 [00:03<00:00, 235.68it/s]

Matching frames:  93%|█████████▎| 897/961 [00:03<00:00, 233.41it/s]

Matching frames: 100%|██████████| 961/961 [00:04<00:00, 231.52it/s]


Evaluating FN edges: 100%|██████████| 161280/161280 [00:00<00:00, 338746.96it/s]


Metrics3 for all file pairs: [{'version': '0.1.1', 'results': {'AOGM': 1761057.0, 'fp_nodes': 330406, 'fn_nodes': 116664, 'ns_nodes': 0, 'fp_edges': 7850, 'fn_edges': 170774, 'ws_edges': 0, 'TRA': 0.10465351568457981, 'DET': 0.1248313437547498}, 'matcher': {'name': 'CTC_Centroid_Matcher', 'distance_threshold': 3, 'scale': (0.27, 0.27, 1.5)}, 'metric': {'name': 'CTCMetrics', 'v_weights': {'ns': 5, 'fp': 1, 'fn': 10}, 'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}}, {'version': '0.1.1', 'results': {'AOGM': 1787924.0, 'fp_nodes': 330605, 'fn_nodes': 119005, 'ns_nodes': 0, 'fp_edges': 8570, 'fn_edges': 172466, 'ws_edges': 0, 'TRA': 0.10015752945498835, 'DET': 0.11999131944444441}, 'matcher': {'name': 'CTC_Centroid_Matcher', 'distance_threshold': 3, 'scale': (0.27, 0.27, 1.5)}, 'metric': {'name': 'CTCMetrics', 'v_weights': {'ns': 5, 'fp': 1, 'fn': 10}, 'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}}, {'version': '0.1.1', 'results': {'AOGM': 1871587.0, 'fp_nodes': 337757, 'fn_nodes': 126679, 'ns_n

In [28]:
metrics3_list

[{'version': '0.1.1',
  'results': {'AOGM': 1761057.0,
   'fp_nodes': 330406,
   'fn_nodes': 116664,
   'ns_nodes': 0,
   'fp_edges': 7850,
   'fn_edges': 170774,
   'ws_edges': 0,
   'TRA': 0.10465351568457981,
   'DET': 0.1248313437547498},
  'matcher': {'name': 'CTC_Centroid_Matcher',
   'distance_threshold': 3,
   'scale': (0.27, 0.27, 1.5)},
  'metric': {'name': 'CTCMetrics',
   'v_weights': {'ns': 5, 'fp': 1, 'fn': 10},
   'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}},
 {'version': '0.1.1',
  'results': {'AOGM': 1787924.0,
   'fp_nodes': 330605,
   'fn_nodes': 119005,
   'ns_nodes': 0,
   'fp_edges': 8570,
   'fn_edges': 172466,
   'ws_edges': 0,
   'TRA': 0.10015752945498835,
   'DET': 0.11999131944444441},
  'matcher': {'name': 'CTC_Centroid_Matcher',
   'distance_threshold': 3,
   'scale': (0.27, 0.27, 1.5)},
  'metric': {'name': 'CTCMetrics',
   'v_weights': {'ns': 5, 'fp': 1, 'fn': 10},
   'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}},
 {'version': '0.1.1',
  'results': {'AOGM

In [29]:
metrics6_list

[{'version': '0.1.1',
  'results': {'AOGM': 1474413.0,
   'fp_nodes': 303468,
   'fn_nodes': 89726,
   'ns_nodes': 0,
   'fp_edges': 17614,
   'fn_edges': 170714,
   'ws_edges': 0,
   'TRA': 0.25038741166302303,
   'DET': 0.2980579686422149},
  'matcher': {'name': 'CTC_Centroid_Matcher',
   'distance_threshold': 6,
   'scale': (0.27, 0.27, 1.5)},
  'metric': {'name': 'CTCMetrics',
   'v_weights': {'ns': 5, 'fp': 1, 'fn': 10},
   'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}},
 {'version': '0.1.1',
  'results': {'AOGM': 1528533.5,
   'fp_nodes': 306229,
   'fn_nodes': 94629,
   'ns_nodes': 0,
   'fp_edges': 17413,
   'fn_edges': 172401,
   'ws_edges': 0,
   'TRA': 0.23070591314238553,
   'DET': 0.27516261574074075},
  'matcher': {'name': 'CTC_Centroid_Matcher',
   'distance_threshold': 6,
   'scale': (0.27, 0.27, 1.5)},
  'metric': {'name': 'CTCMetrics',
   'v_weights': {'ns': 5, 'fp': 1, 'fn': 10},
   'e_weights': {'fp': 1, 'fn': 1.5, 'ws': 1}}},
 {'version': '0.1.1',
  'results': {'AOGM